In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ammi-2024-computer-vision/sample_submission_file.csv
/kaggle/input/ammi-2024-computer-vision/random.txt
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-9351.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-430.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-14967.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-14936.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-4047.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-9063.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-9331.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-5329.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-9646.jpg
/kaggle/input/ammi-2024-computer-vision/extraimages/extraimages/extra-image-14267.jpg
/kaggle/input/ammi-2024-comput

In [2]:
#Imports
import os
import sys
import glob
import torch
import timm
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser

In [3]:
data_path = "../input/ammi-2024-computer-vision/"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
train_path = "/kaggle/input/ammi-2024-computer-vision/train/train"
test_path = "/kaggle/input/ammi-2024-computer-vision/test/test"
# extraimage_path = join(data_path, "extraimages/extraimages")

In [6]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([ transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor()])

normalize = transforms.Normalize(mean=mean, std=std)

class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 224, 224), classCategory
    
train_data = CassavaDataset(train_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms) 


validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]


# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_set = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=train_sampler)
test_set = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

train_len = len(train_set)*32
valid_len = len(test_set)*32

In [7]:
# for class_name, class_index in train_data.class_to_idx.items():
#     print(f"Class: {class_name}, Label: {class_index}")

In [8]:
class Classifier(nn.Module):
    def __init__(self, num_classes, weights=models.ResNet34_Weights.DEFAULT):
        super().__init__()
        
        # Load the pre-trained ResNet34 model
        self.resnet = models.resnet34(weights=weights)
        
        # Modify the final fully connected layer to match the number of classes
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

# Example usage:

In [9]:
num_classes = 5
model = Classifier(num_classes)
print(model)

Classifier(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001,momentum= 0.9)

In [11]:
def train(model, criterion, data_loader1, optimizer, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    model = model.to(device)
    # Make sure model is in training mode.
    #model.train()

    model.train()
    for epoch in range(num_epochs):
        total_correct = 0
        running_loss = 0

        # Training step
        for i, (inputs, labels) in enumerate(data_loader1):
            inputs , labels  = inputs.to(device),labels.to(device)

            output = model(inputs)
            #print(output.logits)
            output_idx = torch.argmax(output,dim =-1)
            total_correct +=(labels==output_idx).sum().item()
            optimizer.zero_grad()
            loss = criterion(output, labels)
            running_loss+=loss.item()*inputs.size(0)
            loss.backward()
            optimizer.step()

        # Print out progress at the end of epoch.
        print(f"Epoch {epoch}, Train Loss: {running_loss/train_len}, Accuracy: {(total_correct/train_len)*100}%")
    print("Finished Training")
    
    

In [12]:
def test(model, test_set):

    with torch.no_grad():
        model.eval()
        total_loss = 0
        total_correct = 0

        # Testing step
        for inputs, labels in test_set:
            inputs , labels  = inputs.to(device),labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss +=loss.item()*inputs.size(0)
            output_idx = torch.argmax(outputs,dim =-1)
            total_correct +=(labels==output_idx).sum().item()

        print(f"Accuracy: {(total_correct/valid_len)*100}%  Loss: {total_loss/valid_len}")

    

In [13]:
num_epochs = 20
train(model, criterion, train_set, optimizer, num_epochs=num_epochs)




Epoch 0, Train Loss: 0.9094042034268799, Accuracy: 65.97711267605634%
Epoch 1, Train Loss: 0.629168664669277, Accuracy: 77.2887323943662%
Epoch 2, Train Loss: 0.5443664183315466, Accuracy: 80.1056338028169%
Epoch 3, Train Loss: 0.5004872249990282, Accuracy: 82.13028169014085%
Epoch 4, Train Loss: 0.45085122776021, Accuracy: 83.84683098591549%
Epoch 5, Train Loss: 0.4414305621233415, Accuracy: 84.17693661971832%
Epoch 6, Train Loss: 0.4158897472506153, Accuracy: 84.92517605633803%
Epoch 7, Train Loss: 0.393517202382881, Accuracy: 85.40933098591549%
Epoch 8, Train Loss: 0.3838431708593394, Accuracy: 86.39964788732394%
Epoch 9, Train Loss: 0.3776090868591318, Accuracy: 86.72975352112677%
Epoch 10, Train Loss: 0.3651479512492431, Accuracy: 86.31161971830986%
Epoch 11, Train Loss: 0.3334926702606846, Accuracy: 87.9181338028169%
Epoch 12, Train Loss: 0.3250945440172152, Accuracy: 88.18221830985915%
Epoch 13, Train Loss: 0.3188146345225543, Accuracy: 88.05017605633803%
Epoch 14, Train Loss: 0

In [19]:
test(model,test_set)


Accuracy: 83.33333333333334%  Loss: 0.48423856295024353


In [15]:
train_data.classes

['cmd', 'cbb', 'cbsd', 'healthy', 'cgm']

In [20]:
# Make submission here
sample_path = "/kaggle/input/ammi-2024-computer-vision/sample_submission_file.csv"
import pandas as pd

def test_mode(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    preds = []
    model.eval()
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, _ in data_loader:
          
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            preds.append(pred.cpu().data.numpy())
            
    return preds

preds = test_mode(model, test_loader)
preds = [item.item() for sublist in preds for item in sublist]

# Extract filenames from the ImageFolder object
name = [os.path.basename(img_path) for _, _, img_path in test_data.file_list]
sample = pd.read_csv(sample_path)
mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}
new_preds = [mapping[pred] for pred in preds]
sample['Category'] = name
sample['Id'] = new_preds
sample["Category"],sample["Id"] = sample["Id"],sample["Category"]


sample.to_csv('submission7.csv', index=False)
sample.head()

,Category,Id
0,cgm,test-img-1448.jpg
1,cmd,test-img-768.jpg
2,cmd,test-img-3481.jpg
3,cmd,test-img-1475.jpg
4,cgm,test-img-2498.jpg


In [17]:
# sample_path ="/kaggle/input/ammi-2024-computer-vision/sample_submission_file.csv"
# #sample_path = os.path.join(sample_path, "sample_submission_file.csv")
# import pandas as pd
# from tqdm import tqdm
# def predict(model, loader):
#     test_dataloader = loader
#     preds = []

#     for test_images, _ in tqdm(test_dataloader):
#         test_images = test_images.to(device)
#         #test_labels = test_labels.to(device)

#         output = model(test_images)

#         _, predicted = torch.max(output, 1)
#         preds.extend(predicted.cpu().data.numpy())


#     return preds

# name = [test_data.file_list[i][-1].split('/')[-1] for i in range(len(test_data.file_list)) ]
# sample = pd.read_csv(sample_path)
# sample['Id'] = name
# preds = predict(model, test_loader)
# mapping = {0: 'cgm', 1: 'cmd', 2: 'healthy', 3: 'cbb', 4: 'cbsd'}
# new_preds = [mapping[pred] for pred in preds]
# sample['Category'] = new_preds
# sample.to_csv('submission_resnet34.csv', index=False)
# sample.head()

In [18]:
2**8

256